# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Question 1
limit = 5
generator = square_root_generator(limit)

sum = 0
for sqrt_value in generator:
    sum += sqrt_value
    print(sqrt_value)

print(f"Sum: {sum}")

print(f"-----------------")

# Question 2
limit = 13
generator = square_root_generator(limit)

sum = 0
for sqrt_value in generator:
    sum += sqrt_value
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
Sum: 8.382332347441762
-----------------
1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [2]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [3]:
# install dlt
%%capture
!pip install dlt[duckdb] # Install dlt with all the necessary DuckDB dependencies

In [4]:
import dlt
import duckdb

# define the connection to load to.
# We now use duckdb as the destination
pipeline = dlt.pipeline(destination='duckdb', dataset_name='ppl_demo')

# run the pipeline with default settings, and capture the outcome
info = pipeline.run(people_1(),
										table_name="people",
										write_disposition="append"
                    )

# show the outcome
print(info)

info = pipeline.run(people_2(),
										table_name="people",
										write_disposition="append"
                    )

print(info)

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")


# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('\n\nLoaded tables: ')
display(conn.sql("show tables"))


Pipeline dlt_colab_kernel_launcher load step completed in 0.47 seconds
1 load package(s) were loaded to destination duckdb and into dataset ppl_demo
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707762048.164599 is LOADED and contains no failed jobs
Pipeline dlt_colab_kernel_launcher load step completed in 0.36 seconds
1 load package(s) were loaded to destination duckdb and into dataset ppl_demo
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707762049.069788 is LOADED and contains no failed jobs


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘

In [5]:
print("\n\n\n People table below:")
people = conn.sql("SELECT * FROM people").df()
display(people)




 People table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707762048.164599,3eD2ZwzkIHJeyA,None
1,2,Person_2,27,City_A,1707762048.164599,q8ovP8U2AECIyQ,None
2,3,Person_3,28,City_A,1707762048.164599,E9/Y0hFMxWFA4Q,None
3,4,Person_4,29,City_A,1707762048.164599,YqNfl46IGpPRnw,None
4,5,Person_5,30,City_A,1707762048.164599,nPW7lLKApsYSuw,None
5,3,Person_3,33,City_B,1707762049.069788,auyp8SorU7cShg,Job_3
6,4,Person_4,34,City_B,1707762049.069788,IPiBGg8DBQl1PQ,Job_4
7,5,Person_5,35,City_B,1707762049.069788,Y0i0TrzkZfBleA,Job_5
8,6,Person_6,36,City_B,1707762049.069788,SUCkeh24DsaYFQ,Job_6
9,7,Person_7,37,City_B,1707762049.069788,o6qhJ1/BrhwwGg,Job_7


In [6]:
# Sum ages of all people
sum_age = conn.sql("SELECT SUM(age) FROM people")
display(sum_age)

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      353 │
└──────────┘

# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [7]:

# run the pipeline with default settings, and capture the outcome
info = pipeline.run(people_1(),
										table_name="people_merge",
										write_disposition="merge",
                    primary_key="ID"
                    )

# show the outcome
print(info)

info = pipeline.run(people_2(),
										table_name="people_merge",
										write_disposition="merge",
                    primary_key="ID"
                    )

print(info)

#conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('\n\nLoaded tables: ')
display(conn.sql("show tables"))

print("\n\n\n people_merge table below:")
people_merge = conn.sql("SELECT * FROM people_merge").df()
display(people_merge)


Pipeline dlt_colab_kernel_launcher load step completed in 0.45 seconds
1 load package(s) were loaded to destination duckdb and into dataset ppl_demo
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707762049.9614527 is LOADED and contains no failed jobs
Pipeline dlt_colab_kernel_launcher load step completed in 0.44 seconds
1 load package(s) were loaded to destination duckdb and into dataset ppl_demo
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707762050.8309562 is LOADED and contains no failed jobs


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people_merge        │
└─────────────────────┘




 people_merge table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,2,Person_2,27,City_A,1707762049.9614527,dIryGSuKn6QXYg,None
1,1,Person_1,26,City_A,1707762049.9614527,jw+FAIbrjPsCEQ,None
2,5,Person_5,35,City_B,1707762050.8309562,qymKkF/K25TcAQ,Job_5
3,4,Person_4,34,City_B,1707762050.8309562,+twHEi2UE/ZhIA,Job_4
4,3,Person_3,33,City_B,1707762050.8309562,lSlSPVDgGZ6cxQ,Job_3
5,6,Person_6,36,City_B,1707762050.8309562,hgmxX7J8Zox5oQ,Job_6
6,8,Person_8,38,City_B,1707762050.8309562,ntLD8148L+GVuA,Job_8
7,7,Person_7,37,City_B,1707762050.8309562,yHOD6tOFnPyukA,Job_7


In [8]:
# Sum ages of all people in merged table
sum_age_merged = conn.sql("SELECT SUM(age) FROM people_merge")
display(sum_age_merged)

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      266 │
└──────────┘

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX